In [1]:
import numpy as np
from pynq import Overlay, allocate
from time import time

In [ ]:
overlay = Overlay("ntt_new.bit")

In [ ]:
dma = overlay.axi_dma_0  

In [ ]:

in_data = np.arange(512, dtype=np.uint16)
in_data=[i+1 for i in in_data]

in_buf  = allocate(shape=(512,), dtype=np.uint32)
out_buf = allocate(shape=(512,), dtype=np.uint32)

in_buf[:] = in_data

In [ ]:

start_time=time()
dma.sendchannel.transfer(in_buf)
dma.recvchannel.transfer(out_buf)
dma.sendchannel.wait()              
dma.recvchannel.wait()              
end_time=time()

In [ ]:

print("Input: ", in_buf)
print("NTT Output:", out_buf[:])


Input:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 24

In [27]:
hw_time=end_time-start_time
print(f'Hardware Time = {hw_time} sec')

Hardware Time = 0.004048824310302734 sec


In [18]:
import numpy as np
from time import time


First 8 output values: [239, 4814, 1228, 3417, 1212, 4290, 6624, 1829]
Time taken: 0.055737 seconds


In [22]:

MOD = 7681
ROOT = 7146
N = 512


In [23]:
def mulmod(a, b):
    return (a * b) % MOD

def addmod(a, b):
    res = a + b
    return res - MOD if res >= MOD else res

def submod(a, b):
    res = a - b
    return res + MOD if res < 0 else res

def modpow(x, e):
    res = 1
    while e > 0:
        if e & 1:
            res = mulmod(res, x)
        x = mulmod(x, x)
        e >>= 1
    return res



In [24]:
def bit_reverse(x, logn):
    res = 0
    for i in range(logn):
        if x & (1 << i):
            res |= 1 << (logn - 1 - i)
    return res



In [ ]:
def ntt(a):
#     logN = N.bit_length() - 1
    logN=9
    
    a = [a[bit_reverse(i, logN)] for i in range(N)]

    # Cooley-Tukey NTT
    len_ = 2
    while len_ <= N:
        wlen = modpow(ROOT, N // len_)
        for i in range(0, N, len_):
            w = 1
            for j in range(len_ // 2):
                u = a[i + j]
                v = mulmod(a[i + j + len_ // 2], w)
                a[i + j] = addmod(u, v)
                a[i + j + len_ // 2] = submod(u, v)
                w = mulmod(w, wlen)
        len_ <<= 1
    return a


In [27]:
input_data = [i % MOD for i in range(N)]

start_time = time()
output_data = ntt(input_data.copy())
end_time = time()

print("First 8 output values:", output_data[:8])
print("Time taken: {:.6f} seconds".format(end_time - start_time))
sw_time=end_time-start_time

First 8 output values: [239, 4814, 1228, 3417, 1212, 4290, 6624, 1829]
Time taken: 0.065758 seconds


In [29]:
speedup=sw_time/hw_time
print(speedup)

10.40906517718987
